In [62]:
import pandas as pd
import itertools

# Define variables
mn = 75
adj_mn = mn * 0.8
mx = 862
adj_mx = mx * 1.35
mx1 = mx * .5
strike_num = 4
diff = mx - mn
a = diff / strike_num
b = a*0.7
c = b*0.7

# Create X1 DataFrame
X1 = pd.DataFrame({'X1': range(int(mn), int(adj_mx), 5)})




In [63]:
adj_mn

60.0

In [64]:
mx1

431.0

In [65]:
X1

,X1
0,75
1,80
2,85
3,90
4,95
...,...
213,1140
214,1145
215,1150
216,1155


In [66]:
def generate_strike_combinations(X1):
    for x1, x2, x3, x4 in itertools.product(X1['X1'], repeat=strike_num):
        if (x1 < x2 < x3 < x4) and (x1 < mx1) and (x4 > mx1) and  ((x3 - x2) >= (x2 - x1)*0.99) and ((x4 - x3) >= (x3 - x2)*0.99) and ((x2 - x1) >= (x3 - x2) >= (x4 - x3)):
            yield {'X1': x1, 'X2': x2, 'X3': x3, 'X4': x4}

# Create strike DataFrame using generator
strike = pd.DataFrame(generate_strike_combinations(X1))

In [67]:
strike

,X1,X2,X3,X4
0,75,195,315,435
1,75,200,325,450
2,75,205,335,465
3,75,210,345,480
4,75,215,355,495
...,...,...,...,...
3499,430,650,870,1090
3500,430,655,880,1105
3501,430,660,890,1120
3502,430,665,900,1135


In [68]:
df=pd.read_excel('Sample_index_data.xlsx',sheet_name='final')

In [69]:
df.columns

Index(['District', 'ID', 'Year', 'Index'], dtype='object')

In [70]:
df=df.sort_values(['District', 'ID', 'Year'])

In [71]:
df['District'].unique()

array(['HIMACHAL PRADESH_Kangra'], dtype=object)

In [72]:
wt=pd.read_excel('Sample_index_data.xlsx',sheet_name='wt')

In [73]:
wt['Geotag Area'].sum()

4890

In [74]:
wt

,District,ID,Geotag Area,wt
0,HIMACHAL PRADESH_Kangra,76.75_31.75,6,0.001227
1,HIMACHAL PRADESH_Kangra,76_31.75,45,0.009202
2,HIMACHAL PRADESH_Kangra,76.25_31.75,354,0.072393
3,HIMACHAL PRADESH_Kangra,76.5_31.75,164,0.033538
4,HIMACHAL PRADESH_Kangra,76.75_32,251,0.051329
5,HIMACHAL PRADESH_Kangra,77_32,5,0.001022
6,HIMACHAL PRADESH_Kangra,75.75_32,134,0.027403
7,HIMACHAL PRADESH_Kangra,76_32,407,0.083231
8,HIMACHAL PRADESH_Kangra,75.5_32,9,0.001840
9,HIMACHAL PRADESH_Kangra,76.25_32,824,0.168507


In [75]:
wt['wt']=wt['Geotag Area']/4890

In [76]:
wt=wt[['District', 'ID','wt']]

In [77]:
df.columns

Index(['District', 'ID', 'Year', 'Index'], dtype='object')

In [78]:
wt.columns

Index(['District', 'ID', 'wt'], dtype='object')

In [79]:
strike['District']='HIMACHAL PRADESH_Kangra'

In [80]:
df.columns

Index(['District', 'ID', 'Year', 'Index'], dtype='object')

In [81]:
import numpy as np
final_data = []

# Iterate over each row in the second dataframe
for index, row in strike.iterrows():
    # Extract 'ID', 'X1', 'X2', 'X3', values for each row
    District = row['District']
    x1 = row['X1']
    x2 = row['X2']
    x3 = row['X3']
    x4 = row['X4']
    
    # Filter the rows from the first dataframe based on the 'District'
    filtered_rows = df[df['District'] == District].copy()  # Copy the filtered rows
    
    # Apply the logic to calculate 'Brn' for the current row
    filtered_rows['Brn'] = np.where(filtered_rows['Index'] < x1, 0, np.nan)
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= x1) & (filtered_rows['Index'] < x2), 10, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= x2) & (filtered_rows['Index'] < x3), 25, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= x3) & (filtered_rows['Index'] < x4), 60, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where(filtered_rows['Index'] >= x4, 100, filtered_rows['Brn'])
    
    values_to_count = [10, 25, 60, 100]
    results = pd.DataFrame()
    for value in values_to_count:
        value_counts = filtered_rows[filtered_rows['Brn'] == value].groupby(['District']).size().reset_index(name=f'Count_{value}')
        if results.empty:
            results = value_counts
        else:
            results = results.merge(value_counts, on=['District'], how='outer')
            
    results = results.fillna(0)
    
    # Ensure all Count columns exist
    for col in ['Count_10', 'Count_25', 'Count_60', 'Count_100']:
        if col not in results.columns:
            results[col] = 0
    
    results['tot'] = results['Count_10'] + results['Count_25'] + results['Count_60'] + results['Count_100']
    id_cnt = filtered_rows.groupby('District')['ID'].nunique().reset_index()
    id_cnt.columns = ['District', 'grd_cnt']
    results = pd.merge(results, id_cnt)
    results['freq'] = results['tot'] / results['grd_cnt']
    
    # Calculate the mean of 'Brn' for different time periods (last 10, 20, 30 rows)
    avg10 = filtered_rows.groupby(['District', 'ID']).tail(10).groupby(['District', 'ID'])['Brn'].mean().reset_index()
    avg20 = filtered_rows.groupby(['District', 'ID']).tail(20).groupby(['District', 'ID'])['Brn'].mean().reset_index()
    avg30 = filtered_rows.groupby(['District', 'ID']).tail(30).groupby(['District', 'ID'])['Brn'].mean().reset_index()
                                    
    z = pd.concat([avg10, avg20, avg30])
    z = z.sort_values(['District', 'ID'])
    z1 = z.groupby(['District', 'ID']).mean().reset_index()
    z2 = pd.merge(z1, wt)
    z2['weigh_avg'] = z2['Brn'] * z2['wt']
    wAvg = z2.groupby(['District'])['weigh_avg'].sum()
    wAvg = wAvg.iloc[0]
    
    # Append 'District', 'X1', 'X2', 'X3', 'X4', 'wAvg', and results columns to the final_data list for each row
    final_data.append([
        District, 
        x1, 
        x2, 
        x3, 
        x4, 
        wAvg, 
        results.iloc[0]['Count_10'], 
        results.iloc[0]['Count_25'], 
        results.iloc[0]['Count_60'], 
        results.iloc[0]['Count_100'], 
        results.iloc[0]['tot'], 
        results.iloc[0]['grd_cnt'], 
        results.iloc[0]['freq']
    ])

# Create a DataFrame from the final_data list with appropriate column names
final_df = pd.DataFrame(final_data, columns=[
    'District', 'X1', 'X2', 'X3', 'X4', 'Brn', 'Count_10', 'Count_25', 'Count_60', 'Count_100', 'tot', 'grd_cnt', 'freq'
])

# Display or return the resulting DataFrame with 'District', 'X1', 'X2', 'X3', 'X4', 'Brn', and results values for each row
print(final_df)

                     District   X1   X2   X3    X4       Brn  Count_10   
0     HIMACHAL PRADESH_Kangra   75  195  315   435  5.477068       220  \
1     HIMACHAL PRADESH_Kangra   75  200  325   450  5.344939       223   
2     HIMACHAL PRADESH_Kangra   75  205  335   465  5.299506       225   
3     HIMACHAL PRADESH_Kangra   75  210  345   480  5.294939       226   
4     HIMACHAL PRADESH_Kangra   75  215  355   495  5.229329       227   
...                       ...  ...  ...  ...   ...       ...       ...   
3499  HIMACHAL PRADESH_Kangra  430  650  870  1090  0.000738         0   
3500  HIMACHAL PRADESH_Kangra  430  655  880  1105  0.000738         0   
3501  HIMACHAL PRADESH_Kangra  430  660  890  1120  0.000738         0   
3502  HIMACHAL PRADESH_Kangra  430  665  900  1135  0.000738         0   
3503  HIMACHAL PRADESH_Kangra  430  670  910  1150  0.000738         0   

      Count_25  Count_60  Count_100    tot  grd_cnt       freq  
0         23.0       4.0        1.0  248.0    

In [114]:
final_df

,District,X1,X2,X3,X4,Brn,Count_10,Count_25,Count_60,Count_100,tot,grd_cnt,freq
0,HIMACHAL PRADESH_Kangra,75,195,315,435,5.477068,220,23.0,4.0,1.0,248.0,22,11.272727
1,HIMACHAL PRADESH_Kangra,75,200,325,450,5.344939,223,21.0,3.0,1.0,248.0,22,11.272727
2,HIMACHAL PRADESH_Kangra,75,205,335,465,5.299506,225,19.0,3.0,1.0,248.0,22,11.272727
3,HIMACHAL PRADESH_Kangra,75,210,345,480,5.294939,226,18.0,3.0,1.0,248.0,22,11.272727
4,HIMACHAL PRADESH_Kangra,75,215,355,495,5.229329,227,18.0,2.0,1.0,248.0,22,11.272727
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,HIMACHAL PRADESH_Kangra,430,650,870,1090,0.000738,0,1.0,0.0,0.0,1.0,22,0.045455
3500,HIMACHAL PRADESH_Kangra,430,655,880,1105,0.000738,0,1.0,0.0,0.0,1.0,22,0.045455
3501,HIMACHAL PRADESH_Kangra,430,660,890,1120,0.000738,0,1.0,0.0,0.0,1.0,22,0.045455
3502,HIMACHAL PRADESH_Kangra,430,665,900,1135,0.000738,0,1.0,0.0,0.0,1.0,22,0.045455


In [111]:
b=final_df[final_df['Brn'].between(2.75,3.5)]

In [112]:
b=b[b['freq'].between(5.5,7)]

In [113]:
b

,District,X1,X2,X3,X4,Brn,Count_10,Count_25,Count_60,Count_100,tot,grd_cnt,freq
292,HIMACHAL PRADESH_Kangra,105,215,325,435,3.204130,125,17.0,3.0,1.0,146.0,22,6.636364
293,HIMACHAL PRADESH_Kangra,105,220,335,450,3.163673,127,15.0,3.0,1.0,146.0,22,6.636364
294,HIMACHAL PRADESH_Kangra,105,225,345,465,3.163673,127,15.0,3.0,1.0,146.0,22,6.636364
295,HIMACHAL PRADESH_Kangra,105,230,355,480,3.055289,128,15.0,2.0,1.0,146.0,22,6.636364
296,HIMACHAL PRADESH_Kangra,105,235,365,495,2.983646,129,15.0,1.0,1.0,146.0,22,6.636364
297,HIMACHAL PRADESH_Kangra,105,240,375,510,2.953704,131,13.0,1.0,1.0,146.0,22,6.636364
298,HIMACHAL PRADESH_Kangra,105,245,385,525,2.952170,132,12.0,1.0,1.0,146.0,22,6.636364
299,HIMACHAL PRADESH_Kangra,105,250,395,540,2.952000,133,11.0,1.0,1.0,146.0,22,6.636364
300,HIMACHAL PRADESH_Kangra,105,255,405,555,2.856567,133,12.0,0.0,1.0,146.0,22,6.636364
301,HIMACHAL PRADESH_Kangra,105,260,415,570,2.817644,134,11.0,0.0,1.0,146.0,22,6.636364


In [24]:
a=final_df[final_df['Brn'].between(2.75,3.5)]
a=a[a['freq'].between(5.5,7.5)]

In [25]:
#a=a[a['Count_60']==1]

In [26]:
a

,District,X1,X2,X3,X4,Brn,Count_10,Count_25,Count_60,Count_100,tot,grd_cnt,freq
466,HIMACHAL PRADESH_Kangra,100,205,310,415,3.388270,134,17.0,5.0,1.0,157.0,22,7.136364
467,HIMACHAL PRADESH_Kangra,100,210,320,430,3.356266,135,18.0,3.0,1.0,157.0,22,7.136364
468,HIMACHAL PRADESH_Kangra,100,215,330,445,3.328829,136,17.0,3.0,1.0,157.0,22,7.136364
469,HIMACHAL PRADESH_Kangra,100,220,340,460,3.288372,138,15.0,3.0,1.0,157.0,22,7.136364
470,HIMACHAL PRADESH_Kangra,100,225,350,475,3.250199,138,16.0,2.0,1.0,157.0,22,7.136364
471,HIMACHAL PRADESH_Kangra,100,230,360,490,3.178556,139,16.0,1.0,1.0,157.0,22,7.136364
472,HIMACHAL PRADESH_Kangra,100,235,370,505,3.108345,140,15.0,1.0,1.0,157.0,22,7.136364
473,HIMACHAL PRADESH_Kangra,100,240,380,520,3.078403,142,13.0,1.0,1.0,157.0,22,7.136364
474,HIMACHAL PRADESH_Kangra,100,245,390,535,3.076869,143,12.0,1.0,1.0,157.0,22,7.136364
475,HIMACHAL PRADESH_Kangra,100,250,400,550,3.076698,144,11.0,1.0,1.0,157.0,22,7.136364


In [306]:
a=a[a['freq'].between(5.5,7)]

In [307]:
a

,District,X1,X2,X3,X4,Brn,Count_10,Count_25,Count_60,Count_100,tot,grd_cnt,freq
167,HIMACHAL PRADESH_Kangra,110,210,310,410,2.752585,108,16.0,5.0,1.0,130.0,22,5.909091


In [149]:
a.to_excel('sample_op.xlsx')

In [142]:
import pandas as pd
import numpy as np

# Assuming 'df' is the first dataframe and 'strike' is the second dataframe

# Create an empty list to store the results
final_data = []

# Iterate over each row in the second dataframe
for index, row in strike.iterrows():
    # Extract 'ID', 'X1', 'X2', 'X3', values for each row
    District = row['District']
    x1 = row['X1']
    x2 = row['X2']
    x3 = row['X3']
    x4 = row['X4']
    
    # Filter the rows from the first dataframe based on the 'ID'
    filtered_rows = df[df['District'] == District].copy()  # Copy the filtered rows
    
    # Apply the logic to calculate 'Brn' for the current row
    filtered_rows['Brn'] = np.where(filtered_rows['Index'] < x1, 0, np.nan)
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= x1) & (filtered_rows['Index'] < x2), 10, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= x2) & (filtered_rows['Index'] < x3), 25, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= x3) & (filtered_rows['Index'] < x4), 60, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where(filtered_rows['Index'] >= x4, 100, filtered_rows['Brn'])
    
    values_to_count = [10, 25, 60, 100]
    results = pd.DataFrame()
    for value in values_to_count:
        value_counts = filtered_rows[filtered_rows['Brn'] == value].groupby(['District']).size().reset_index(name=f'Count_{value}')
        if results.empty:
            results = value_counts
        else:
            results = results.merge(value_counts, on=['District'], how='outer')
            
    results = results.fillna(0)
    results['tot'] = results['Count_10'] + results['Count_25'] + results['Count_60'] + results['Count_100']
    id_cnt = filtered_rows.groupby('District')['ID'].nunique().reset_index()
    id_cnt.columns = ['District', 'grd_cnt']
    results = pd.merge(results, id_cnt)
    results['freq'] = results['tot'] / results['grd_cnt']
    
    # Calculate the mean of 'Brn' for different time periods (last 10, 20, 30 rows)
    avg10 = filtered_rows.groupby(['District', 'ID']).tail(10).groupby(['District', 'ID'])['Brn'].mean().reset_index()
    avg20 = filtered_rows.groupby(['District', 'ID']).tail(20).groupby(['District', 'ID'])['Brn'].mean().reset_index()
    avg30 = filtered_rows.groupby(['District', 'ID']).tail(30).groupby(['District', 'ID'])['Brn'].mean().reset_index()
                                    
    z = pd.concat([avg10, avg20, avg30])
    z = z.sort_values(['District', 'ID'])
    z1 = z.groupby(['District', 'ID']).mean().reset_index()
    z2 = pd.merge(z1, wt)
    z2['weigh_avg'] = z2['Brn'] * z2['wt']
    wAvg = z2.groupby(['District'])['weigh_avg'].sum()
    wAvg = wAvg.iloc[0]
    
    # Append 'District', 'X1', 'X2', 'X3', 'X4', 'wAvg', and results columns to the final_data list for each row
    final_data.append([
        District, 
        x1, 
        x2, 
        x3, 
        x4, 
        wAvg, 
        results.iloc[0]['Count_10'], 
        results.iloc[0]['Count_25'], 
        results.iloc[0]['Count_60'], 
        results.iloc[0]['Count_100'], 
        results.iloc[0]['tot'], 
        results.iloc[0]['grd_cnt'], 
        results.iloc[0]['freq']
    ])

# Create a DataFrame from the final_data list with appropriate column names
final_df = pd.DataFrame(final_data, columns=[
    'District', 'X1', 'X2', 'X3', 'X4', 'Brn', 'Count_10', 'Count_25', 'Count_60', 'Count_100', 'tot', 'grd_cnt', 'freq'
])

# Display or return the resulting DataFrame with 'District', 'X1', 'X2', 'X3', 'X4', 'Brn', and results values for each row
print(final_df)


KeyError: 'Count_10'

In [124]:
final_df.head(5)

,District,X1,X2,X3,X4,Brn,Count_10,Count_25,Count_60,Count_100,tot,grd_cnt,freq
0,HIMACHAL PRADESH_Kangra,60,160,260,360,7.954272,260,46.0,10.0,2,318.0,22,14.454545
1,HIMACHAL PRADESH_Kangra,60,185,285,360,7.251068,281,28.0,7.0,2,318.0,22,14.454545
2,HIMACHAL PRADESH_Kangra,60,185,285,385,7.251068,281,28.0,7.0,2,318.0,22,14.454545
3,HIMACHAL PRADESH_Kangra,60,185,310,410,6.938133,281,31.0,5.0,1,318.0,22,14.454545
4,HIMACHAL PRADESH_Kangra,60,185,310,435,6.938133,281,31.0,5.0,1,318.0,22,14.454545


In [58]:
a=final_df[final_df['Brn'].between(2.75,3.25)]

In [25]:
a.to_excel('Himachal_op.xlsx')

In [26]:
a.shape

(23046, 6)

In [229]:
filtered_rows=df

In [230]:
    filtered_rows['Brn'] = np.where(filtered_rows['Index'] < 110, 0, np.nan)
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= 110) & (filtered_rows['Index'] < 210), 10, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= 210) & (filtered_rows['Index'] < 310), 25, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= 310) & (filtered_rows['Index'] < 410), 60, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where(filtered_rows['Index'] >= 410, 100, filtered_rows['Brn'])

In [232]:
filtered_rows.sort_values(['Index'])

,District,ID,Year,Index,Brn
55,HIMACHAL PRADESH_Kangra,76_31.75,2019,4.40,0.0
454,HIMACHAL PRADESH_Kangra,75.5_32.25,1998,5.92,0.0
161,HIMACHAL PRADESH_Kangra,77_32,2005,5.94,0.0
548,HIMACHAL PRADESH_Kangra,76.75_32.5,2002,6.05,0.0
549,HIMACHAL PRADESH_Kangra,76.75_32.5,2003,6.58,0.0
...,...,...,...,...,...
644,HIMACHAL PRADESH_Kangra,76.25_32.5,2008,316.24,60.0
480,HIMACHAL PRADESH_Kangra,76.25_32.25,1994,349.04,60.0
331,HIMACHAL PRADESH_Kangra,76.75_32.25,1995,355.89,60.0
494,HIMACHAL PRADESH_Kangra,76.25_32.25,2008,401.58,60.0


In [23]:
    values_to_count = [10, 25, 60, 100]
    results = pd.DataFrame()
    for value in values_to_count:
        value_counts = filtered_rows[filtered_rows['Brn'] == value].groupby(['District']).size().reset_index(name=f'Count_{value}')
        if results.empty:
            results = value_counts
        else:
            results = results.merge(value_counts, on=['District'], how='outer')
            
    results = results.fillna(0)
    results['tot'] = results['Count_10'] + results['Count_25'] + results['Count_60'] + results['Count_100']
    id_cnt = filtered_rows.groupby('District')['ID'].nunique().reset_index()
    id_cnt.columns = ['District', 'grd_cnt']
    results = pd.merge(results, id_cnt)
    results['freq'] = results['tot'] / results['grd_cnt']

In [24]:
results

,District,Count_10,Count_25,Count_60,Count_100,tot,grd_cnt,freq
0,HIMACHAL PRADESH_Kangra,128,15,2,1,146,22,6.636364


In [25]:
    avg10 = filtered_rows.groupby(['District', 'ID']).tail(10).groupby(['District', 'ID'])['Brn'].mean().reset_index()
    avg20 = filtered_rows.groupby(['District', 'ID']).tail(20).groupby(['District', 'ID'])['Brn'].mean().reset_index()
    avg30 = filtered_rows.groupby(['District', 'ID']).tail(30).groupby(['District', 'ID'])['Brn'].mean().reset_index()
                                    
    z = pd.concat([avg10, avg20, avg30])
    z = z.sort_values(['District', 'ID'])
    z1 = z.groupby(['District', 'ID']).mean().reset_index()
    z2 = pd.merge(z1, wt)
    z2['weigh_avg'] = z2['Brn'] * z2['wt']
    wAvg = z2.groupby(['District'])['weigh_avg'].sum()
    wAvg = wAvg.iloc[0]

In [26]:
    final_data.append([
        District, 
        x1, 
        x2, 
        x3, 
        x4, 
        wAvg, 
        results.iloc[0]['Count_10'], 
        results.iloc[0]['Count_25'], 
        results.iloc[0]['Count_60'], 
        results.iloc[0]['Count_100'], 
        results.iloc[0]['tot'], 
        results.iloc[0]['grd_cnt'], 
        results.iloc[0]['freq']
    ])

In [28]:
results.iloc[0]['Count_10']

128

In [228]:
filtered_rows=df

In [ ]:
    filtered_rows['Brn'] = np.where(filtered_rows['Index'] < x1, 0, np.nan)
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= x1) & (filtered_rows['Index'] < x2), 10, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= x2) & (filtered_rows['Index'] < x3), 25, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= x3) & (filtered_rows['Index'] < x4), 60, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where(filtered_rows['Index'] >= x4, 100, filtered_rows['Brn'])
    
    values_to_count = [10, 25, 60, 100]
    results = pd.DataFrame()
    for value in values_to_count:
        value_counts = filtered_rows[filtered_rows['Brn'] == value].groupby(['District']).size().reset_index(name=f'Count_{value}')
        if results.empty:
            results = value_counts
        else:
            results = results.merge(value_counts, on=['District'], how='outer')
            
    results = results.fillna(0)
    
    # Ensure all Count columns exist
    for col in ['Count_10', 'Count_25', 'Count_60', 'Count_100']:
        if col not in results.columns:
            results[col] = 0
    
    results['tot'] = results['Count_10'] + results['Count_25'] + results['Count_60'] + results['Count_100']
    id_cnt = filtered_rows.groupby('District')['ID'].nunique().reset_index()
    id_cnt.columns = ['District', 'grd_cnt']
    results = pd.merge(results, id_cnt)
    results['freq'] = results['tot'] / results['grd_cnt']
    
    # Calculate the mean of 'Brn' for different time periods (last 10, 20, 30 rows)
    avg10 = filtered_rows.groupby(['District', 'ID']).tail(10).groupby(['District', 'ID'])['Brn'].mean().reset_index()
    avg20 = filtered_rows.groupby(['District', 'ID']).tail(20).groupby(['District', 'ID'])['Brn'].mean().reset_index()
    avg30 = filtered_rows.groupby(['District', 'ID']).tail(30).groupby(['District', 'ID'])['Brn'].mean().reset_index()
                                    
    z = pd.concat([avg10, avg20, avg30])
    z = z.sort_values(['District', 'ID'])
    z1 = z.groupby(['District', 'ID']).mean().reset_index()
    z2 = pd.merge(z1, wt)
    z2['weigh_avg'] = z2['Brn'] * z2['wt']
    wAvg = z2.groupby(['District'])['weigh_avg'].sum()
    wAvg = wAvg.iloc[0]